In [ ]:
# I suggest to install them directly on terminal
!pip install tensorflow==2.3.4
!pip install tensorflow-io

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
import tensorflow_io as tfio
from keras.callbacks import EarlyStopping
from PIL import Image

2025-12-22 09:12:52.575403: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-22 09:12:53.178742: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-22 09:12:56.156581: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-22 09:12:56.209796: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-22 09:13:02.183378: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
inference_directory = 'inference/'

def validate_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()
            except (IOError, SyntaxError):
                print(f"Invalid image file removed: {file_path}")
                os.remove(file_path)

validate_images(inference_directory)

In [3]:
crowd_model = tf.keras.models.load_model('crowd_model.keras')

In [4]:
inference_ds = tf.keras.preprocessing.image_dataset_from_directory(
    inference_directory, labels='inferred', label_mode='int', image_size=(256, 256), 
    validation_split=None, subset=None)

mean_confidence = 0
correct = 0
test_size = 0

for batch_num, (X, Y) in enumerate(inference_ds):
    batch_size = len(Y)
    test_size += batch_size
    pred = crowd_model.predict(X)
    for i in range(batch_size):
        mean_confidence += np.max(pred[i], axis=-1)
        predicted = np.argmax(pred[i], axis=-1)
        actual = Y[i]
        if predicted == actual:
            correct += 1

mean_confidence /= test_size

print(f'Number correct: {correct} out of {test_size}')
print(f'Accuracy {correct / test_size}')
print(f'Mean Confidence Score {mean_confidence}')

Found 6 files belonging to 2 classes.
1/1 [==============================] - 1s 810ms/step
Number correct: 6 out of 6
Accuracy 1.0
Mean Confidence Score 0.9314993421236674
